# Build a Simple LLM Application with LCEL

```LCEL - LangChain Expression Language```

This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, a great way to get started with LangChain - a lot of features can be built with just prompting and an LLM call!

This emphasizes on : 
- Using language models.
- Using PromptTemplates and OutputParsers
- Using LangChain Expression Language (LCEL) to chain components together.
- Debugging and tracing your application using LangSmith
- Deploying your application with LangServe

In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
groq_api_key = os.getenv("GROQ_API_KEY")

In [8]:
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
model = ChatGroq(model="llama-3.1-8b-instant",groq_api_key=groq_api_key)

In [9]:
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000001CB0DCC3110>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001CB0DCC3B10>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [13]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello buddy, how are you?")
]

response = model.invoke(messages)

In [15]:
response.content

"Bonjour l'ami, comment vas-tu ?"

In [16]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
parser.invoke(response)

"Bonjour l'ami, comment vas-tu ?"

In [17]:
### Using LCEL - we can chain the components

chain = model | parser
chain.invoke(messages)


'Salut camarade, comment vas-tu ?'

In [18]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template = "Translate the following into {language}:"

prompt = ChatPromptTemplate.from_messages([
    ("system", generic_template),
    ("user", "{text}")
])

In [20]:
result = prompt.invoke({
    "language": "French",
    "text": "How was your day?"
})

In [22]:
result.to_messages()

[SystemMessage(content='Translate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='How was your day?', additional_kwargs={}, response_metadata={})]

In [ ]:
chain = prompt | model | parser
chain.invoke({
    "language": "French",
    "text": "Hello, how was your day?"
})

'Bonjour, comment était votre journée ?'